# Filtering spiking data

In this notebook we simulate the effect of applying MLSpike to the traces generated by our spike trains. In order to do so, we follow Huang et al., eLife 2021. Here, the autors report the probabilities of detecting an action potential, given the amount of events in a bin of 250ms. 

In order to use this notebook, one first needs to generate a simulated HDF5 file. For more instructions on how to do so, please refer to the `spiking_neuron_model` notebook.

In [ ]:
# The autoreload extension allows you to tweak the code in the imported modules
# and rerun cells to reflect the changes.
%load_ext autoreload
%autoreload 2
# this allows us to output the dictionary structure
#%load_ext ipy_dict_hierarchy
# matplotlib settings to look decent in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import sys
# the helpers are in the `/ana/` folder.
sys.path.append("./../")
from ana import ana_helper as ah
from ana import plot_helper as ph
from ana import paper_plots as ppl

#Libraries to generate results from the notebook itself.
import numpy as np
import pandas as pd
from benedict import benedict
import matplotlib.pyplot as plt

The values below are the measured probabilities obtained from Figure 7 of Huang et al. These where obtained just by analysing the figure with dedicated software. The cases correspond to the MLSpike bars, and different experimental conditions. Which one is used for filtering is indicated later as an argument

In [ ]:
#Values of probabilities from the ALife paper have been scrapped and they are hardcoded here
#for the different conditions
prob_values = {}
prob_values["hr_emxs"] = np.array([0.0958, 0.373, 0.575, 0.695, 0.811])
prob_values["hr_tetos"] = np.array([0.194, 0.553, 0.821, 0.7913, 1.0])
prob_values["hr_cuxf"] = np.array([0.096, 0.280, 0.533, 0.568, 0.698])
prob_values["hr_emxf"] = np.array([0.054, 0.396, 0.580, 0.752, 0.916])

prob_values["lr_emxs"] = np.array([0.052, 0.236, 0.423, 0.592, 0.700])
prob_values["lr_tetos"] = np.array([0.106, 0.469, 0.722, 0.939, 1.0])
prob_values["lr_cuxf"] = np.array([0.081, 0.329, 0.649, 0.759, 0.897])
prob_values["lr_emxf"] = np.array([0.049, 0.349, 0.521, 0.624, 0.690])

## Auxiliary functions

These functions are in charge of binning raster plot and performing the filtering from the given spike train

In [ ]:
def bin_spikes(spiketimes, bintime=0.25, tf=1200):
    """
    Given an array of spike times, produces a binned timeseries, as a well as an nd-array which contains
    the index of the bin to which each spike time belongs to.
    Parameters
    ==========
    - spiketimes: ndarray
        A NxK array which has the spike times (in seconds) for the all neurons. 
        N is number of neurons, and K the maximum number of spikes. 
        Non-used spaces of the array are coded with NaNs.
    - bintime: float
        Time width of the bin to be used, in seconds. Default is 250ms.
    - tf: float
        Total simulation time, in seconds
    Returns
    =======
    - binned: numpy array
        1-d array that contains the amount of spikes in each bin, as integers.
    - which_bin: ndarray
        An array with shape NxK which instead of having spiketimes contains the index
        of the correspondent bins. 
    """
    
    #Gerate the bins
    bins = np.arange(0, tf, bintime)
    
    #Here we don't care which neuron generated the spike. Just delete nans.
    data = spiketimes.flatten() 
    data = data[~np.isnan(data)]
    
    #Histogram produces the binned activity timeseries. Digitize is a handy function to get in
    #which bin everything was
    return np.histogram(data, bins=bins, density=False)[0], np.digitize(spiketimes, bins=bins)


def prob_2_use(spikecount, prob_detection):
    """
    Selects which probability will be used in each bin, depending on the number of action potential
    in such bins.
    Parameters
    ==========
    - spikecount: ndarray
        A NxK array which has the spike times (in seconds) for the all neurons. 
        See "bin_spikes" docs for more info.
    - prob_detection: ndarray
        1-dim array with 5 elements that contains the probability P(k) of detecting 1 AP if there 
        is k spikes in the bins
    Returns
    =======
    - probs: ndarray
        An array which contains the probability of detection for each time bin.
    """
    
    #By default, probability of detection is 1
    values = np.ones(spikecount.size)
    
    #If there was nothing, nothing to do; 
    #If there is 6+ spikes, assume prob=100%
    mask = (spikecount <= 5) & (spikecount > 0)
    
    #Apply mask
    values[mask] = prob_detection[spikecount[mask] - 1] 
    return values

def spikes_not_detected(binned, prob_values, prob_table="hr_emxs"):
    """
    Returns a new array with the spike count after the filtering is applied.
    Parameters
    ==========
    - binned: ndarray
        Contains the total number of spikes in each bin
    - prob_values: dict
        This dictionary contains the probability P(k) of detecting 1 spike if there 
        is k spikes in the bins for different experimental cases. Which case is selected
        depends on "prob_table" argument. Default is "hr_emxs".
    - p_table: string
        To select one entry from the prob_values dictionary. Possible values are shown in the
        filter_spikes notebook
    Returns
    =======
    - detected_spikes: ndarray
        Contains the detected number of spikes in each bin after the filter is applied
    """  
    #Get the probabilities to use in each bin
    probs = prob_2_use(binned, prob_values[prob_table])
    #Generate values according to a binomial distribution with "n=binned, p=probs" for each bin.
    return np.random.binomial(binned, probs) 


def delete_from_spikeseries(binned, masked, which_bin, spiketimes):
    """
    Given the actual amount of spikes and the result after filtering, erase randomly 
    some action potentials from the spiketimes, without modyfying it.
    Parameters
    ==========
    - binned: ndarray
        Contains the total number of spikes in each bin
    - masked: ndarray
        Contains the detected number of spikes in each bin after the filter is applied
    - which_bin: ndarray
        An array with shape NxK which instead of having spiketimes contains the index
        of the correspondent bins.
    - spiketimes: ndarray
        A NxK array which has the spike times (in seconds) for the all neurons. 
        See "bin_spikes" docs for more info.
    Returns
    =======
    - new_spiketimes: ndarray
        Same as spiketimes, but with some randomly deleted spikes to match the masked array. 
        These are marked with a "-1" value
    """  
    
    #Initialize the arrays
    corrected = np.zeros(binned.size)
    n_neurons, n_times = spiketimes.shape
    new_spikes = spiketimes.copy()
    
    #Loop on times and after that in neurons, to delete spikes "vertically" in the raster
    for j in range(n_times):
        for i in range(n_neurons):
            #In which bin is this spiketime?
            index = which_bin[i,j] - 1 
            
            #Nans are coded as 4800, so everything that is 4799 is not valid
            if index < binned.size-1:
                #binned- masked = number of deleted spikes. So keep deleting
                #until these two are equal
                if masked[index] + corrected[index] < binned[index]:
                    corrected[index] += 1
                    new_spikes[i, j] = -1
    return new_spikes


def prepare_file(path):
    # quick helper to do preprocessing matching the usual settings
    bs_large = 20 / 1000 # ms
    threshold_factor = 2.5 / 100 # %
    remove_null_sequences = False

    h5f = ah.prepare_file(path, hot=False)
    ah.find_rates(h5f, bs_large=bs_large)
    ah.find_system_bursts_from_global_rate(
        h5f,
        rate_threshold=threshold_factor * np.nanmax(h5f["ana.rates.system_level"]),
        merge_threshold=0.1,
        skip_sequences=False,
    )

    # this is a global setting for now
    if remove_null_sequences:
        ah.remove_bursts_with_sequence_length_null(h5f)

    ah.find_ibis(h5f)
    ah.find_participating_fraction_in_bursts(h5f)
    ah.find_isis(h5f)

    return h5f

## Filtering

Use the functions defined above to process the data. After that, the spiketrain inside the variable `h5f_new` is substituted by the filtered one, and graphs are produced.

# Effect of filtering on burst observables

Now we do check the effect of the filtering in some of the system observables. To do so, we first run the function to find rates, which are subsequently used to find bursts and then IBIs. These are plotted for both cases.

In [ ]:
h5f = prepare_file("/Users/paul/Library/Mobile Documents/com~apple~CloudDocs/para/2_Projects/modular_cultures/_repo/_latest/dat.nosync/simulations/lif/raw/stim=02_k=3_kin=30_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_stimrate=0.0_rep=001.hdf5")

h5f_new = h5f.clone()


# do the filtering
spiketrain = h5f["data"]["spiketimes"]
#Bin the raster plot
binned, which_bin = bin_spikes(spiketrain)
#Apply the filtering and get its spike count
masked = spikes_not_detected(binned, prob_values, prob_table="lr_emxs")
#Use this to delete spikes 
new_spikes = delete_from_spikeseries(binned, masked, which_bin, spiketrain)
#Apply the new spiketrain to the copy of the H5F file
h5f_new["data"]["spiketimes"] = new_spikes

# lets have a look:
h5f

In [ ]:
#Convert results into DataFrame, as expected by the plot violins function

#Repeat for a new DF for bursts. 
df_sl = pd.DataFrame()
#Compute the sequence length to see number of modules participating in a burst
es_old = [len(seq) for seq in h5f["ana.bursts.system_level.module_sequences"]]
es_new = [len(seq) for seq in h5f_new["ana.bursts.system_level.module_sequences"]]
df_sl["Sequence Length"] = np.concatenate([es_old, es_new])
df_sl["Filter"] = ["Full" for k in range(len(es_old))] + ["Filtered" for k in range(len(es_new))]

#Repeat for IBIs
df_ibis = pd.DataFrame()
ibis_old = h5f["ana.ibi.system_level.any_module"]
ibis_new = h5f_new["ana.ibi.system_level.any_module"]
df_ibis["IBIs"] = np.concatenate([ibis_old, ibis_new])
df_ibis["Filter"] = ["Full" for k in range(len(ibis_old))] + ["Filtered" for k in range(len(ibis_new))]

#Repeat for a new DF of event sizes
df_es = pd.DataFrame()
es_old = h5f["ana.bursts.system_level.participating_fraction"]
es_new = h5f_new["ana.bursts.system_level.participating_fraction"]
df_es["Event size"] = np.concatenate([es_old, es_new])
df_es["Filter"] = ["Full" for k in range(len(es_old))] + ["Filtered" for k in range(len(es_new))]

# Correlation
rij = ah.find_rij(h5f, time_bin_size=500/1000)
np.fill_diagonal(rij, np.nan)
rij = rij.flatten()

rij_new = ah.find_rij(h5f_new, time_bin_size=500/1000)
np.fill_diagonal(rij_new, np.nan)
rij_new = rij_new.flatten()

df_rij = pd.DataFrame()
df_rij["Correlation"] = np.concatenate([rij, rij_new])
df_rij["Filter"] = ["Full" for k in range(len(rij))] + ["Filtered" for k in range(len(rij_new))]

In [ ]:
# Plot all together!!

# Paper
fig, axes = plt.subplot_mosaic(
    """
AAAA
BBBB
CDEF
""",
    figsize=(6, 3),
    gridspec_kw=dict(height_ratios=[0.75, 0.75, 1]),
)


ph.plot_raster(h5f, axes["A"], zorder=-3)
axes["A"].set_xlabel("")
ph.plot_raster(h5f_new, axes["B"], zorder=-3)

for lab in "AB":
    axes[lab].set_xlim(100, 406)

palette = dict(Full="black", Filtered="gray")

ppl.custom_violins(
    df_sl, category="Filter", observable="Sequence Length", ax=axes["C"], palette=palette
)
ppl.custom_violins(
    df_es, category="Filter", observable="Event size", ax=axes["D"], palette=palette
)
ppl.custom_violins(
    df_ibis, category="Filter", observable="IBIs", ax=axes["E"], palette=palette
)
ppl.custom_violins(
    df_rij, category="Filter", observable="Correlation", ax=axes["F"], palette=palette
)

axes["C"].set_ylabel("Nubmer of modules\nparticipating in events")
axes["C"].set_ylim(1, 4)
axes["D"].set_ylabel("Event size")
axes["D"].set_ylim(0, 1)
axes["E"].set_ylabel("Inter-event interval\n(seconds)")
axes["E"].set_ylim(0, None)
axes["F"].set_ylabel("Neuron correlation")
axes["F"].set_ylim(0, 1)

for lab in "CDEF":
    axes[lab].set_xlabel("")
    ppl._noclip(axes[lab])
    ppl.sns.despine(ax=axes[lab], bottom=False, offset = 2)

plt.tight_layout()
# plt.savefig("effect_filter.pdf")
plt.show()
